# Transformer Chatbot


Implemented by Suyoung Kim, MLCL, 2023.02.13 (persistence7388@gmail.com)

In [18]:
! pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import pandas as pd
import numpy as np
import re
import torch 
import sentencepiece as spm
import sys
from tqdm import tqdm
from torch.nn import Transformer
from torch import nn
import math
from torch.utils.data import Dataset, DataLoader

In [20]:
#Hyperparameter
MAX_LENGTH = 40
BATCH_SIZE = 64
lr = 1e-4
embed_size = 256 
n_head=8
n_hid = 512
n_layer = 2
dropout = 0.1
epoch = 30

#seed
random_seed = 9712
torch.manual_seed(random_seed)
torch.backends.cudnn.enabled = False
print(torch.randn(1, 3))

tensor([[0.3936, 0.5584, 0.9692]])


In [21]:
#GPU
! nvidia-smi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mon Feb 13 02:17:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    32W /  70W |    648MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
# Data 
train_data = pd.read_csv('https://raw.githubusercontent.com/Doheon/Chatbot-Transformer/main/ChatBotData.csv')

questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)


answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

with open('all.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(questions))
    f.write('\n'.join(answers))


corpus = "all.txt"
prefix = "chatbot"
vocab_size = 16000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    "--min_frequency={3}"+
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

vocab_file = "chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "안녕하세요 만나서 반갑습니다"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)


print(line)
print(pieces)
print(ids)
print(vocab.GetPieceSize())
vocab_size = vocab.GetPieceSize()

안녕하세요 만나서 반갑습니다
['▁안녕하세요', '▁만나서', '▁반갑습니다']
[4626, 1930, 8499]
16007


In [23]:

START_TOKEN = [2]
END_TOKEN = [3]

#tokenize and padding  

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  for (sentence1, sentence2) in zip(inputs, outputs):
    zeros1 = np.zeros(MAX_LENGTH, dtype=int)
    zeros2 = np.zeros(MAX_LENGTH, dtype=int)
    sentence1 = START_TOKEN + vocab.encode_as_ids(sentence1) + END_TOKEN
    zeros1[:len(sentence1)] = sentence1[:MAX_LENGTH]

    sentence2 = START_TOKEN + vocab.encode_as_ids(sentence2) + END_TOKEN
    zeros2[:len(sentence2)] = sentence2[:MAX_LENGTH]

    tokenized_inputs.append(zeros1)
    tokenized_outputs.append(zeros2)
  return tokenized_inputs, tokenized_outputs

questions_encode, answers_encode = tokenize_and_filter(questions, answers)
print(questions_encode[0])
print(answers_encode[0])

[    2  5566 14968  3210   111     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[   2 5192  217 5936    7    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [24]:

class MyDataset(Dataset):
    def __init__(self, questions, answers):
        questions = np.array(questions)
        answers = np.array(answers)
        self.inputs = questions
        self.dec_inputs = answers[:,:-1] #(datanum, max_len-1)
        self.outputs = answers[:,1:] #(datanum, max_len-1) 
        self.length = len(questions) #input_length 
    
    def __getitem__(self,idx):
        return (self.inputs[idx], self.dec_inputs[idx], self.outputs[idx])

    def __len__(self):
        return self.length


dataset = MyDataset(questions_encode, answers_encode)
dataloader = DataLoader(dataset, shuffle=True, batch_size=BATCH_SIZE)
print(f"data set num: {len(dataset)}")
print(f"data set num: {len(dataloader)}")

data set num: 11823
data set num: 185


In [26]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, n_head, n_hid, n_layer, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.transformer = Transformer(embed_size, n_head, dim_feedforward=n_hid, num_encoder_layers=n_layer, num_decoder_layers=n_layer,dropout=dropout)
        self.e_pos = PositionalEncoding(embed_size, dropout)
        self.e_embedding = nn.Embedding(vocab_size, embed_size)
        self.d_pos = PositionalEncoding(embed_size, dropout)
        self.encoder_d = nn.Embedding(vocab_size, embed_size)
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.linear = nn.Linear(embed_size, vocab_size)
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.e_embedding.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.e_embedding(src) * math.sqrt(self.embed_size) #(batch_size, max_len, embed_size)
        src = self.e_pos(src) #(batch_size, max_len, embed_size)
        tgt = self.encoder_d(tgt) * math.sqrt(self.embed_size)#(batch_size, max_len-1, embed_size)
        tgt = self.d_pos(tgt)#(batch_size, max_len-1, embed_size)
        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask) #(max_len-1, batch_size,embed_size)
        output = self.linear(output) #???
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask
    
model = TransformerModel(vocab_size, embed_size=embed_size, n_head=n_head, n_hid=n_hid, n_layer=n_layer, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [27]:


model.train()
for i in range(epoch):
    batchloss = 0.0
    progress = tqdm(dataloader)
    for (inputs, dec_inputs, outputs) in progress:
        optimizer.zero_grad()
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device) #(max_len, max_len)
        src_padding_mask = gen_attention_mask(inputs).to(device) #(batch_size, max_len)
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)  #(max_len-1, max_len-1)
        tgt_padding_mask = gen_attention_mask(dec_inputs).to(device) #(batch_size, max_len-1)
        result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask) #(max_len, batch_size, vocab_size)
        loss = criterion(result.permute(1,2,0), outputs.to(device).long())
        loss.backward()
        optimizer.step()
        batchloss += loss
        progress.set_description("{:0.3f}".format(loss))
    print("epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(dataloader))

1.007: 100%|██████████| 185/185 [00:15<00:00, 11.74it/s]


epoch: 1 | loss: 1.7898976918813345


0.886: 100%|██████████| 185/185 [00:13<00:00, 14.19it/s]


epoch: 2 | loss: 0.9690366177945524


0.962: 100%|██████████| 185/185 [00:13<00:00, 14.15it/s]


...

epoch: 28 | loss: 0.37386474609375


0.397: 100%|██████████| 185/185 [00:13<00:00, 14.11it/s]


epoch: 29 | loss: 0.35309072958456506


0.227: 100%|██████████| 185/185 [00:13<00:00, 14.14it/s]

epoch: 30 | loss: 0.3333302265888936


In [ ]:
torch.save(model.state_dict(), "chatbot.pth")

In [ ]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output = torch.tensor([START_TOKEN]).to(device)

    model.eval()
    for i in range(MAX_LENGTH):
        #mask 
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).to(device)
        src_padding_mask = gen_attention_mask(input).to(device)
        tgt_padding_mask = gen_attention_mask(output).to(device)

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1) #(batch_size, output_size, vocab_size)

        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(END_TOKEN[0])):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.to(device)], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size])))
    
    print("========================================")
    print('Q: {}'.format(sentence))
    print('A: {}'.format(predicted_sentence))


In [ ]:
model.load_state_dict(torch.load("chatbot.pth"))
result = predict("놀고싶다")
result = predict("감기 같애")
result = predict("건강하게 다이어트 하는 방법")
result = predict("게임하고 싶어")
result = predict("궁금하지?")

Q: 놀고싶다
A: 저도 데려가세요 .
Q: 감기 같애
A: 병원가세요 .
Q: 건강하게 다이어트 하는 방법
A: 오늘 일찍 주무세요 .
Q: 게임하고 싶어
A: 지금도 늦지 않았어요 .
Q: 궁금하지?
A: 개인의 선택이죠 .
